### Import Data

In [28]:
import json

# importing data if in json format
def load_json_data(file: str) -> dict:
    try:
        with open(file, "r", encoding="utf-8") as f:
            data = json.load(f)
        return data
    except FileNotFoundError:
        print(f"File '{file}' not found.")
        return {}
    except json.JSONDecodeError:
        print(f"Error decoding JSON data from file '{file}'.")
        return {}


# get all txt files from Data Dump folder #TODO: move this to a util file
import os
import numpy as np
import subprocess

def load_txt_data():
    data_points = []
    if os.path.exists(file_path):
        for file in os.listdir("Data Dump"):
            if file.endswith(".txt"):
                with open(os.path.join("Data Dump", file), "r", encoding="utf-8") as f:
                    content = f.read()
                    data_points.append(content)
        return np.array(data_points)
    else: 


def  get_recent_query_data():
    # Define the repository URL and the file path
    repo_url = "https://github.com/username/repo"
    file_path = "/path/to/local/file.txt"

    # Clone the repository to a temporary directory
    temp_dir = "/tmp/repo"
    subprocess.run(["git", "clone", repo_url, temp_dir])

    # Move the file from the temporary directory to the desired location
    subprocess.run(["mv", f"{temp_dir}/file.txt", file_path])

    # Remove the temporary directory
    subprocess.run(["rm", "-rf", temp_dir])


def load_txt_data(repo_url: str, file_path: str):
    data_points = []
    
    # Check if the file exists locally
    if os.path.exists(file_path):
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()
            data_points.append(content)
    else:
        # Download the file from the repository
        try:
            response = requests.get(repo_url)
            if response.status_code == 200:
                content = response.text
                data_points.append(content)
                
                # Save the downloaded file locally
                with open(file_path, "w", encoding="utf-8") as f:
                    f.write(content)
            else:
                print(f"Failed to download file from {repo_url}.")
        except requests.exceptions.RequestException as e:
            print(f"Error downloading file from {repo_url}: {e}")
    
    return np.array(data_points)

## Data Prep

### Search

In [8]:
# install the following if necessary
!pip install --use-pep517 annoy
!pip install ipywidgets
# if not in a virtual environment,
#jupyter nbextension enable --py widgetsnbextension
# if in a virtual environment,
!jupyter nbextension enable --py --sys-prefix widgetsnbextension

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 5.9 MB/s eta 0:00:00m eta 0:00:010:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp39-cp39-linux_x86_64.whl size=75070 sha256=5f63e7a64ff7cef577f23c2cde7b3acfca43e7280fadca74a2a6603119557a46
  Stored in directory: /home/mkelbessa/.cache/pip/wheels/09/a9/54/37478e65995fe712f7da465749da9ddb21db6b1a599d591ac7
Successfully built annoy
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 2.4 MB/s eta 0:00:0031m2.6 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.9/214.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 11.0 MB/s eta 0:00:00 MB/s eta 0:00:01:01
  Attempting uninstall: comm
    Found existing installation: comm 0.1.2
    Uninstalling comm-0.1.2:
      Successfully uninstalled comm-0.

In [27]:
import json
import numpy as np
import pandas as pd 
from tqdm import tqdm
from annoy import AnnoyIndex
from functions.preprocessing import *
from multiprocessing import Pool, cpu_count
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)

QA_MODEL_NAME = "multi-qa-distilbert-cos-v1"


# load model 
def load_model():
    model = SentenceTransformer(QA_MODEL_NAME)
    return model

# there seems to be a bug with the model, so we need to reload it in case it all the initial encoding fails
model = load_model()

def test_encode(model):
    sentence = """
    Amidst the verdant tapestry of the ancient forest, where sunlight dapples the forest floor and the rustling leaves whisper secrets of generations past, a curious menagerie 
    of creatures, from the tiniest iridescent insects to the towering, majestic oak trees,coexists in a harmonious symphony of life, while the distant echoes of a 
    bubbling brook weave a soothing melody through the very heart of this enchanting wilderness.
    """
    assert np.all(model.encode(sentences=sentence) != 0)

try:
    test_encode(model)
except Exception as e:
    print("Failed to encode sentence, reloading model")
    model = load_model()

In [ ]:
# list of subreddits to concentrate on 
!wget https://raw.githubusercontent.com/username/repo/main/subreddits.csv -O subreddits.csv

subreddits = pd.read_csv("subreddits.csv").dropna()

# load query data 
query_data = load_data(load_data")

In [ ]:




# sentence = "Amidst the verdant tapestry of the ancient forest, where sunlight dapples the forest floor and the rustling leaves whisper secrets of generations past, a curious menagerie of creatures, from the tiniest iridescent insects to the towering, majestic oak trees, coexists in a harmonious symphony of life, while the distant echoes of a bubbling brook weave a soothing melody through the very heart of this enchanting wilderness."

# test_encode = model.encode(sentences=sentence)

# if not already filtered, load data and clean it 
narrative_df = pd.read_csv('unfiltered_sample_narratives.csv')

# make lowercase 
narrative_df['selftext'] = narrative_df['selftext'].str.lower()

# remove if there are any null values and narratives have less than 50 words
narrative_df = narrative_df.dropna()
narrative_df = narrative_df[narrative_df['selftext'].str.split().str.len().gt(50)]

narrative_df.to_csv("filtered_sample_narratives.csv")



oldnarrative_queries = list(json.load(open("local_data.json", encoding="utf-8"))["File"]) 


##### Lemmatize

In [ ]:
tokenized_queries = list(map(lemmatize, oldnarrative_queries))

In [ ]:
#lemmatize narratives  

num_processes = 11
pool = Pool(processes=num_processes)

results = pool.imap(func=lemmatize,  
                    iterable= narrative_df['selftext'],
                    chunksize= 24370) 

pool.close()
pool.join()
 
narrative_df["tokenized_selftext"] = np.array([result for result in results])
narrative_df.to_csv("Tokenized_filtered_2018-01.csv", index=False)

##### Create Search Queries and Embeddings

In [ ]:
#load lemmatized narratives
tokenized_narratives_df = pd.read_csv("D:\Reddit Dataset\Submissions\Decompressed\csv\RS_2018-01_lemmatized.csv")

In [ ]:
tokenized_narratives_df.shape

In [ ]:
tokenized_narratives_df = tokenized_narratives_df[tokenized_narratives_df["subreddit"].isin(subreddits['subreddit'])]
tokenized_narratives_df.reset_index(drop=True, inplace=True)

In [ ]:
tokenized_narratives_df.shape

In [ ]:
query_embeddings = model.encode(sentences= tokenized_queries,
                                convert_to_numpy=True,
                                show_progress_bar=True, 
                                normalize_embeddings=True)

np.save('old-narrative-queries_embeddings.npy', query_embeddings)

query_search_index = AnnoyIndex(query_embeddings.shape[1], 'manhattan')

for index, embed_value in enumerate(query_embeddings):
    query_search_index.add_item(index, embed_value)

query_search_index.build(n_trees = 20, n_jobs = 3)

query_search_index.save('old-narrative-queries_search_index.ann')

Note: You can either use the CPU or GPU to encode (use gpu if possible as its faster), but don't use both at the same time so as not to do the computation twice.

In [ ]:
# embedding narratives: CPU version
narratives_embedding = model.encode(sentences=tokenized_narratives_df["lemmatized_selftext"], 
                                    convert_to_numpy=True, 
                                    show_progress_bar=True)

np.save('RS_2018-01_narrative_embeddings.npy', narratives_embedding)

In [ ]:
# embedding narratives: GPU version
pool = model.start_multi_process_pool()

narratives_embedding = model.encode_multi_process(sentences=tokenized_narratives_df["lemmatized_selftext"], 
                                                pool=pool)

model.stop_multi_process_pool(pool=pool)

np.save('RS_2018-01_narrative_embeddings.npy', narratives_embedding)

In [ ]:
narratives_search_index = AnnoyIndex(np.array(narratives_embedding).shape[1], 'manhattan')

for index_embedding, embed_value in enumerate(tqdm(narratives_embedding)):
    narratives_search_index.add_item(index_embedding, embed_value)

narratives_search_index.build(n_trees = 20, n_jobs = -1)

narratives_search_index.save(f'RS_2018-01_narrative_search_index.ann')

##### Load embeddings and search indexes

In [ ]:
# load embeddings and search indexes for new narratives 
narratives_embedding = np.load('RS_2018-01_narrative_embeddings.npy')
narratives_search_index = AnnoyIndex(np.array(narratives_embedding).shape[1], 'manhattan')
narratives_search_index.load('RS_2018-01_narrative_search_index.ann')

# oldnarrative based: load embeddings and search indexes queries
query_embeddings = np.load('old-narrative-queries_embeddings.npy')
query_search_index = AnnoyIndex(np.array(query_embeddings).shape[1], 'manhattan')
query_search_index.load('old-narrative-queries_search_index.ann')

##### Simple Concatenating (for a single query)

In [ ]:
#retrive nearest neighbors
results_list = []

for index, query_embedding in tqdm(enumerate(query_embeddings)): 
    similar_item_ids = narratives_search_index.get_nns_by_vector(query_embedding, n=100, include_distances=True)

    result_df = pd.DataFrame(data={
        'selftext': narrative_df['selftext'][similar_item_ids[0]],
        'title': narrative_df['title'][similar_item_ids[0]],
        'ids': narrative_df['id'][similar_item_ids[0]],
        'distance': similar_item_ids[1]
    })

    results_list.append(result_df)

results = pd.concat(results_list)
results = results.drop_duplicates(subset=['selftext'])
results = results.sort_values(by=['distance'], ascending=True)

results.to_csv("results_tokenized_full.csv", index=False)

##### Comparative Concatenating

In [ ]:
def compare(id_to_distance_dictionary: dict, threshold: float) -> float:
    """
    Filters for stories which contain intersecting topics specified by the threshold. 

    :param id_to_distance_dictionary
    :param

    return: 
        tuple containing all the ids for relevant narratives and their respective distances
    """
   
    filtered_id_distance_array = []

    for comparable_key in id_to_distance_dictionary:
        # array we are currently trying to filter
        comparable_id_array = id_to_distance_dictionary[comparable_key][0]
        comparable_distance_array = id_to_distance_dictionary[comparable_key][1]

        for comparable_id_index, comparable_id in enumerate(comparable_id_array):
            percentage = 0
            for tocompare_key in id_to_distance_dictionary:
                if tocompare_key != comparable_key:
                    tocompare_id_array = id_to_distance_dictionary[tocompare_key][0]

                    if tocompare_id_array.__contains__(comparable_id): 
                        percentage += (1 / (len(id_to_distance_dictionary)-1))

            if percentage >= threshold: 
                filtered_id_distance_array.append((comparable_id, comparable_distance_array[comparable_id_index],))

    return filtered_id_distance_array

In [ ]:
similar_dict = {}
for index, query_embedding in tqdm(enumerate(query_embeddings)): 
    similar_item_ids = narratives_search_index.get_nns_by_vector(
                                    query_embedding,
                                    n=1000, 
                                    include_distances=True)

    similar_dict[f"query_{index}"] = similar_item_ids


filterd_arr= compare(similar_dict, threshold= 0.7)

In [ ]:
tokenized_narratives_df.columns

In [ ]:
for index, similar_item_ids in enumerate(filterd_arr): 

    new_row = {
        'selftext': tokenized_narratives_df['selftext'][similar_item_ids[0]],
        'title': tokenized_narratives_df['title'][similar_item_ids[0]],
        'subreddit': tokenized_narratives_df['subreddit'][similar_item_ids[0]],
        'permalink': tokenized_narratives_df['permalink'][similar_item_ids[0]],
        'ids': tokenized_narratives_df['id'][similar_item_ids[0]],
        'distance': similar_item_ids[1]
    }

    if index != 0:
        filtered_results_df.loc[len(filtered_results_df)] = new_row
    else: 
        filtered_results_df = pd.DataFrame([new_row])

filtered_results_df.to_csv("Filtered_Results_RS_2018-01.csv", index=False)

##### Results

In [ ]:
# add empty row to the bottom of the dataframe
import pandas as pd 
results_df = pd.read_csv("Filtered_Results_RS_2018-01.csv")
results_df.reset_index(drop=True)

In [ ]:
results_df = results_df[results_df["distance"] < 20.00]

In [ ]:
results_df.sort_values(by=["distance"])

In [ ]:
from functions.document_writer import docx_writer
from paths import HOME

docx_writer(num_people=5, dataframe= results_df)